In [1]:
# import all libraries
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [2]:
# all used tickers
tickers = ['LLY', 'NVO', 'JNJ', 'MRK', 'ABBV', 'MRNA',
           'NVS', 'AZN', 'PFE', 'AMGN', 'PPH', 'IHE', 'PJP']

dfs = {}

for symbol in tickers:
    # for each symbol, load the pkl file and store it in the dictionary as a df
    dfs[f"{symbol}_df"] = pd.read_csv(f"pharma-data/clean-data/stocks/{symbol}_df.csv", index_col='Date', parse_dates=True)

In [3]:
# read in covid data
covid_df = pd.read_csv("pharma-data/clean-data/us_covid_deaths.csv", index_col='date', parse_dates=True)

In [4]:
# print the first 2 rows of each df
for df_name, df in dfs.items():
    print(df_name + "\n")
    print(df.head(2))
    print("")

LLY_df

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2020-01-06  131.419998  132.559998  130.940002  132.259995  124.553093   
2020-01-07  131.699997  132.929993  131.699997  132.509995  124.788521   

             Volume  
Date                 
2020-01-06  2102900  
2020-01-07  2448300  

NVO_df

                 Open   High        Low      Close  Adj Close   Volume
Date                                                                  
2020-01-06  28.495001  28.59  28.434999  28.504999  26.977007  2495000
2020-01-07  28.500000  28.50  28.280001  28.475000  26.948616  2080600

JNJ_df

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2020-01-06  144.000000  144.199997  142.850006  144.100006  128.432449   
2020-01-07  144.009995  145.449997  141.380005  144.979996  129.216751   

        

In [5]:
# sanity check
covid_df.head()

,US covid deaths
date,
2020-01-06,0.0
2020-01-07,0.0
2020-01-08,0.0
2020-01-09,0.0
2020-01-10,0.0


In [6]:
# prep dict for the merged data
merged_dfs = {}

for df_name, df in dfs.items():
    # convert index to datetime
    df.index = pd.to_datetime(df.index)

    # merge data frames based on date index, using inner join to keep only rows with matching dates. (keeps trading days)
    merged_dfs[df_name] = pd.merge(df, covid_df, how='inner', left_index=True, right_index=True)

In [14]:
# send merged dfs to csv in /merged-dfs

directory = 'pharma-data/merged-dfs'

for df_name, df in merged_dfs.items():
    df.to_csv(f"{directory}/{df_name}-covid.csv")

In [ ]:
# check merge worked
for df_name, df in merged_dfs.items():
    print(df_name + "\n")
    print(df.head())
    print("")

In [ ]:
# check for correlation among columns. 
for df_name, df in merged_dfs.items():
    print(df_name + "\n")
    print(df.corr())
    print("")

In [ ]:
# Find the pearson correlation coefficient among closing price & deaths for the day
for df_name, df in merged_dfs.items():
    p_coef, p_val = stats.pearsonr(df['Close'], df['US covid deaths'])
    print(f"{df_name} Pearson Coef: {p_coef}" + "\n")

In [ ]:
# check for the best fit - linear, quadratic, or cubic between close and US covid deaths
pearson_corr = {}
for df_name, df in merged_dfs.items():
    pearson_corr[df_name] = df['Close'].corr(df['US covid deaths'])

# fit models and calculate r-squared
r_squared_values = {}
for df_name, df in merged_dfs.items():
    x = df['Close'].values.reshape(-1, 1)
    y = df['US covid deaths'].values

    # linear regression
    model_linear = LinearRegression()
    model_linear.fit(x, y)
    y_pred_linear = model_linear.predict(x)
    r_squared_values[f"{df_name}_linear"] = model_linear.score(x, y)

    # quadratic regression
    poly = PolynomialFeatures(degree=2)
    x_poly = poly.fit_transform(x)
    model_quad = LinearRegression()
    model_quad.fit(x_poly, y)
    y_pred_quad = model_quad.predict(x_poly)
    r_squared_values[f"{df_name}_quadratic"] = model_quad.score(x_poly, y)

    # cubic regression
    poly = PolynomialFeatures(degree=3)
    x_poly = poly.fit_transform(x)
    model_cubic = LinearRegression()
    model_cubic.fit(x_poly, y)
    y_pred_cubic = model_cubic.predict(x_poly)
    r_squared_values[f"{df_name}_cubic"] = model_cubic.score(x_poly, y)

# print r-squared values
print("\nR-squared values:")
count = 0
for model_name, r_squared in r_squared_values.items():
    print(f"{model_name}: {r_squared}")
    count+=1

    # after printing the 3 possibilities, print new line
    if count % 3 == 0:
        print("")

In [ ]:
# check example shape
merged_dfs['ABBV_df'].shape